# Day 11

## Part 1

Each monkey has several attributes:

Starting items lists your worry level for each item the monkey is currently holding in the order they will be inspected.
Operation shows how your worry level changes as that monkey inspects an item. (An operation like new = old * 5 means that your worry level after the monkey inspected the item is five times whatever your worry level was before inspection.)
Test shows how the monkey uses your worry level to decide where to throw an item next.
If true shows what happens with an item if the Test was true.
If false shows what happens with an item if the Test was false.
After each monkey inspects an item but before it tests your worry level, your relief that the monkey's inspection didn't damage the item causes your worry level to be divided by three and rounded down to the nearest integer.

The monkeys take turns inspecting and throwing items. On a single monkey's turn, it inspects and throws all of the items it is holding one at a time and in the order listed. Monkey 0 goes first, then monkey 1, and so on until each monkey has had one turn. The process of each monkey taking a single turn is called a round.

When a monkey throws an item to another monkey, the item goes on the end of the recipient monkey's list. A monkey that starts a round with no items could end up inspecting and throwing many items by the time its turn comes around. If a monkey is holding no items at the start of its turn, its turn ends.

Focus on the two most active monkeys if you want any hope of getting your stuff back. Count the total number of times each monkey inspects items over 20 rounds.


In [1]:
# Libraries

import numpy as np
import pandas as pd

# Read input file
all_lines = []

with open('input.txt') as file:
    all_lines = [line.strip() for line in file]

# Add empty string at the end
all_lines.append('')


In [2]:
# Parse lines

from collections import deque

monkeys = []

single_monkey_lines = []
for line in all_lines:
    # Find all lines for single monkey
    if line != '':
        single_monkey_lines.append(line)
    
    else:
        # Parse data for monkey
        
        # Monkey ID
        monkey_id = int(single_monkey_lines[0][7:-1])
        
        # Items
        items = single_monkey_lines[1][16:].split(', ')
        items = deque([int(item) for item in items])
        
        # Operation
        operation_str = single_monkey_lines[2][21:]
        if operation_str == '* old':
            operator = 'power'
            operator_arg = 2
        elif operation_str.split(' ')[0] == '*':
            operator = 'multiply'
            operator_arg = int(operation_str.split(' ')[1])
        
        elif operation_str.split(' ')[0] == '+':
            operator = 'addition'
            operator_arg = int(operation_str.split(' ')[1])
        
        else:
            raise ValueError(f'Unrecognized operation: {operation_str}')
        
        # Test
        divisble_by_int = int(single_monkey_lines[3].split(' ')[-1])
        throw_to_if_true = int(single_monkey_lines[4].split(' ')[-1])
        throw_to_if_false = int(single_monkey_lines[5].split(' ')[-1])
        
        # Store 
        monkey_dict = {
            'id': monkey_id,
            'items': items,
            'operator': operator,
            'operator_arg': operator_arg,
            'divisble_by_int': divisble_by_int,
            'throw_to_if_true': throw_to_if_true,
            'throw_to_if_false': throw_to_if_false
        }
        
        monkeys.append(monkey_dict)
        single_monkey_lines = []
    

In [3]:
# Game logic

from typing import Any, Dict, List

def _calculate_new_worry_level(*, old: int, operator: str, operator_arg: int) -> int:
    if operator == 'addition':
        return old + operator_arg
    elif operator == 'multiply':
        return old * operator_arg
    elif operator == 'power':
        return old**operator_arg
    else:
        raise ValueError(f'Unrecognized operation: {operator}')


def play_round(*, monkeys: List[Dict[str, Any]], monkeys_inspection_count: Dict[int, int],
              ) -> List[Dict[str, Any]]:
    for monkey in monkeys:
        monkey_id = monkey['id']
        while monkey['items']:
            # Get item
            item = monkey['items'].popleft()
            
            # Increase worry level
            item = _calculate_new_worry_level(
                old=item, operator=monkey['operator'], operator_arg=monkey['operator_arg'])
            
            # Relief
            item = int(item/3)
            
            # Test
            if item % monkey['divisble_by_int'] == 0:
                # Throw to monkey
                monkeys[monkey['throw_to_if_true']]['items'].append(item)
            
            else:
                # Throw to monkey
                monkeys[monkey['throw_to_if_false']]['items'].append(item)
            
            # Inspection count
            monkeys_inspection_count[monkey_id] = monkeys_inspection_count[monkey_id] + 1
        
    return monkeys
    


In [4]:
# Play rounds

from copy import deepcopy
from tqdm import tqdm

rounds_monkeys_list = []
monkeys_inspection_count = {x: 0 for x in range(len(monkeys))}

monkeys_game = deepcopy(monkeys)
rounds_monkeys_list.append(monkeys_game)

for round_n in tqdm(range(1, 21)):
    monkeys_game = play_round(
        monkeys=monkeys_game, monkeys_inspection_count=monkeys_inspection_count)
    rounds_monkeys_list.append(deepcopy(monkeys_game))


100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 9858.51it/s]


In [5]:
# Result
item_counts = []
for m, count in monkeys_inspection_count.items():
    item_counts.append(count)
    
max_count1, max_count2 = sorted(item_counts)[-2:]

print(max_count1*max_count2)


101436


---

## Part 2

Worry levels are no longer divided by three after each item is inspected; you'll need to find another way to keep your worry levels manageable. Starting again from the initial state in your puzzle input, what is the level of monkey business after 10000 rounds?


In [6]:
# Observation - working with big numbers is very slow ==> an alternative method needs to be used
# Since throwing test use division, store only modulo for every item:
# -> if mod is 0, then multiplication and power will not change it, similarly the addition
# to the original item will have the same mod as addition to the mod of the original item


In [7]:
# New monkeys list

from copy import deepcopy

monkeys_part2 = deepcopy(monkeys)

# Find common_denominator
common_denominator = 1
for monkey in monkeys_part2:
    common_denominator = common_denominator * monkey['divisble_by_int']
print(f'Common denominator {common_denominator}')


Common denominator 9699690


In [8]:
def play_round_part2(*, monkeys: List[Dict[str, Any]], monkeys_inspection_count: Dict[int, int],
                     common_denominator: int) -> List[Dict[str, Any]]:
    for monkey in monkeys:
        monkey_id = monkey['id']
        while monkey['items']:
            # Get item
            item = monkey['items'].popleft()
            
            # Increase worry level
            item = _calculate_new_worry_level(
                old=item, operator=monkey['operator'], operator_arg=monkey['operator_arg'])
            
            # Test
            modified_item = item % common_denominator
            if modified_item % monkey['divisble_by_int'] == 0:
                # Throw to monkey
                monkeys[monkey['throw_to_if_true']]['items'].append(modified_item)
            
            else:
                # Throw to monkey
                monkeys[monkey['throw_to_if_false']]['items'].append(modified_item)
            
            # Inspection count
            monkeys_inspection_count[monkey_id] = monkeys_inspection_count[monkey_id] + 1
        
    return monkeys


In [9]:
# Play rounds

from copy import deepcopy
from tqdm import tqdm

monkeys_inspection_count_part2 = {x: 0 for x in range(len(monkeys))}

monkeys_game = deepcopy(monkeys)
rounds_monkeys_list.append(monkeys_game)

N_ROUNDS = 10000

for round_n in tqdm(range(1, N_ROUNDS+1)):
    monkeys_game = play_round_part2(
        monkeys=monkeys_game, monkeys_inspection_count=monkeys_inspection_count_part2,
        common_denominator=common_denominator)
    

100%|██████████████████████████████████| 10000/10000 [00:00<00:00, 45813.85it/s]


In [10]:
# Result
item_counts = []
for m, count in monkeys_inspection_count_part2.items():
    item_counts.append(count)
    
max_count1, max_count2 = sorted(item_counts)[-2:]

print(max_count1*max_count2)


19754471646
